In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
%matplotlib inline

# Toy 2-D Interpolator
Assumptions
- Input points are evenly spaced
- Input points are monotonically increasing
- Doesn't take into account edge cases

## Method
Uses the formula for bilinear interpolation from Wikipedia:
$$
f(x,y) = \frac{1}{(x_2 - x_1)(y_2-y_1)} \left[x_2 - x \quad x - x_1\right] \begin{bmatrix}
f(Q_{11}) & f(Q_{12}) \\
f(Q_{21}) & f(Q_{22})
\end{bmatrix} \begin{bmatrix}
y_2 - y \\
y-y_1
\end{bmatrix}
$$
Where the 1 indicates a lower value, and 2 indicates a higher one- this applies to Q, in which each Q is a point indicated by the subscripts. i.e. (Q_11) is x_lo, y_lo


I am also implementing this using OOP, as the Superfish tracking code utilizes RegularGridInterpolator which is also implemented with OOP. This is mainly for ease of compatibility.

In [2]:
class ToyBilinearInterpolator:
    # Create the interpolator with data for interpolation
    def __init__(self, points, data, grid_tolerance=0.05):
        self.x_vals, self.y_vals = points
        self.data = np.array(data)

        # Run some basic checks for the input data
        self.__sanitizeCoordinates();

        # if the data is reasonable, then do the spacing checks
        self.dx = np.diff(self.x_vals).flat
        self.dy = np.diff(self.y_vals).flat

        if not self.__isRegularlySpaced(grid_tolerance):
            print(f"dx: {self.dx[0]},\t dy{self.dy[0]}")
            raise ValueError("Input x and y arrays are not evenly spaced!")

    def __call__(self, point):
        """Calls the bilinear interpolation routine when provided a x,y point."""
        x_vals, y_vals, data = self.x_vals, self.y_vals, self.data
        x, y = point

        # As we are regularly spaced, we can use the simple floor method to
        # get the index of the value closest to the interpolant
        lower_index_x = int((x - x_vals[0])/self.dx[0])
        lower_index_y = int((y - y_vals[0])/self.dy[0])

        # build the f(Q_xy) dudes
        q11 = data[lower_index_x][lower_index_y]
        q12 = data[lower_index_x][lower_index_y + 1]
        q21 = data[lower_index_x + 1][lower_index_y]
        q22 = data[lower_index_x + 1][lower_index_y + 1]

        # and the x1,x2,y1,y2 dudes
        x1 = x_vals[lower_index_x]
        x2 = x_vals[lower_index_x + 1]
        y1 = y_vals[lower_index_y]
        y2 = y_vals[lower_index_y + 1]

        # the expanded form of that matrix equation on top
        return (q11 * (x2 - x) * (y2 - y) +
                q12 * (x2 - x) * (y - y1) +
                q21 * (x - x1) * (y2 - y) +
                q22 * (x - x1) * (y - y1)) / ((x2 - x1) * (y2 - y1))

    def __sanitizeCoordinates(self):
        x_vals, y_vals, data = self.x_vals, self.y_vals, self.data
        """Does basic checks for the input data."""

        # Ensures that there are enough points in the data array to
        # actually do an interpolation (in this case, 2 for x and 2 for y)
        if len(x_vals) < 2 and len(y_vals) < 2:
            raise ValueError("Not enough points for an interpolation!")

        # Ensures that the x and y values for the input data have a value
        if len(x_vals) * len(y_vals) != data.size:
            raise ValueError("Not every x and y point has a value!")

        # Ensures that x and y array is monotonically increasing
        if not all(i<j for i, j in zip(x_vals, x_vals[1:])):
            raise ValueError("The x array is not monotonically increasing!")

        if not all(i<j for i, j in zip(y_vals, y_vals[1:])):
            raise ValueError("The y array is not monotonically increasing!")

    def __isRegularlySpaced(self, tolerance):
        """Checks if the spacing beween the x and y arrays is consistent between
        themselves and each other."""
        dx, dy = self.dx, self.dy

        # Check if the x and y values are evenly spaced
        if (np.allclose(dx, dx[0], rtol=tolerance) and np.allclose(dy, dy[0], rtol=tolerance)):
            # and that the x and y values are evenly spaced from each other
            return np.allclose(dx[0], dy[0], rtol=tolerance)
        else:
            return False

# Basic Testing

In [3]:
interp = ToyBilinearInterpolator(
    points=((1, 2, 3, 4, 5),
            (1, 2, 3, 4, 5)),
    data=((110, 120, 130, 140, 150),
          (210, 220, 230, 240, 250),
          (310, 320, 330, 340, 350),
          (410, 420, 430, 440, 450),
          (510, 520, 530, 540, 550)
    ))

print('test 1')
print(interp((3, 3))) # should be 330

print("\ntest 2")
print(interp((1.25, 1.25))) # should be 137.5

test 1
330.0

test 2
137.5


# Superfish Testing

In [4]:
f_res = 2.856e9
phi_RF = 0 #np.pi
Amp_fac=0.12

In [5]:
from scipy.interpolate import RegularGridInterpolator


def trueSuperfishFields(fname):
    with open(fname) as fin:
        lines = fin.readlines()
        linesplit = [lines[i].split() for i in range(len(lines))]
        Zmin = float(linesplit[0][0])/100
        Zmax = float(linesplit[0][1])/100
        Nz = int(linesplit[0][2])
        freq = float(linesplit[1][0])*1e6
        Rmin = float(linesplit[2][0])/100
        Rmax = float(linesplit[2][1])/100
        Nr = int(linesplit[2][2])

    rr = np.linspace(Rmin,Rmax,Nr+1,endpoint=True)
    zz = np.linspace(Zmin,Zmax,Nz+1,endpoint=True)

    print(Zmin, Zmax, Nz, Rmin, Rmax, Nr)

    E = np.zeros([Nz+1,Nr+1,2])
    Emag = np.zeros([Nz+1,Nr+1])
    Bt = np.zeros([Nz+1,Nr+1])

    linesplitE = linesplit[4::2]
    linesplitB = linesplit[5::2]

    for ind in range(len(linesplitE)):
        i = ind % (Nz+1)
        j = int(ind / (Nz+1)) - 1
        E[i,j,0] = float(linesplitE[ind][0])*1e6
        E[i,j,1] = float(linesplitE[ind][1])*1e6
        Emag[i,j] = float(linesplitE[ind][2])*1e6
        Bt[i,j] = float(linesplitB[ind][0])*4e-7*np.pi

    ## Chris -- Here's a little assignment to work on when you get a second
    # The 'RegularGridInterpolator' function in scipy basically takes discrete data, i.e E[:,:,0/1]
    # and converts it into a function that can spit out the right fields at any (z,r).
    # Write a custom function that does this using only basic numpy operations and python primitives.
    # We're eventually going to use numba to speed it up (and parallelize with MPI/CUDA down the line).
    ERint = RegularGridInterpolator((zz,rr),E[:,:,1])
    EZint = RegularGridInterpolator((zz,rr),E[:,:,0])
    BTint = RegularGridInterpolator((zz,rr),Bt[:,:])

    return ERint,EZint,BTint, rr, zz, Emag

    
def toySuperfishFields(fname):
    with open(fname) as fin:
        lines = fin.readlines()
        linesplit = [lines[i].split() for i in range(len(lines))]
        Zmin = float(linesplit[0][0])/100
        Zmax = float(linesplit[0][1])/100
        Nz = int(linesplit[0][2])
        freq = float(linesplit[1][0])*1e6
        Rmin = float(linesplit[2][0])/100
        Rmax = float(linesplit[2][1])/100
        Nr = int(linesplit[2][2])

    rr = np.linspace(Rmin,Rmax,Nr+1,endpoint=True)
    zz = np.linspace(Zmin,Zmax,Nz+1,endpoint=True)

    print(Zmin, Zmax, Nz, Rmin, Rmax, Nr)

    E = np.zeros([Nz+1,Nr+1,2])
    Emag = np.zeros([Nz+1,Nr+1])
    Bt = np.zeros([Nz+1,Nr+1])

    linesplitE = linesplit[4::2]
    linesplitB = linesplit[5::2]

    for ind in range(len(linesplitE)):
        i = ind % (Nz+1)
        j = int(ind / (Nz+1)) - 1
        E[i,j,0] = float(linesplitE[ind][0])*1e6
        E[i,j,1] = float(linesplitE[ind][1])*1e6
        Emag[i,j] = float(linesplitE[ind][2])*1e6
        Bt[i,j] = float(linesplitB[ind][0])*4e-7*np.pi

    ## Chris -- Here's a little assignment to work on when you get a second
    # The 'RegularGridInterpolator' function in scipy basically takes discrete data, i.e E[:,:,0/1]
    # and converts it into a function that can spit out the right fields at any (z,r).
    # Write a custom function that does this using only basic numpy operations and python primitives.
    # We're eventually going to use numba to speed it up (and parallelize with MPI/CUDA down the line).
    ERint = ToyBilinearInterpolator((zz,rr),E[:,:,1])
    EZint = ToyBilinearInterpolator((zz,rr),E[:,:,0])
    BTint = ToyBilinearInterpolator((zz,rr),Bt[:,:])

    return ERint,EZint,BTint, rr, zz, Emag

In [6]:
import math

def Exint(r):
    r = np.flip(r)
    rad = np.sqrt(r[0]**2 + r[1]**2)
    th = math.atan2(r[1],r[0])
    z = r[-1]

    return [ERint((z,rad))*np.cos(th)]

def Eyint(r):
    r = np.flip(r)
    rad = np.sqrt(r[0]**2 + r[1]**2)
    th = math.atan2(r[1],r[0])
    z = r[-1]

    return [ERint((z,rad))*np.sin(th)]

def Ezint(r):
    r = np.flip(r)
    rad = np.sqrt(r[0]**2 + r[1]**2)
    th = math.atan2(r[1],r[0])
    z = r[-1]

    return [EZint((z,rad))]

def Efld(r,t):
    try:
        E = Amp_fac*np.array([Exint(r)[0],Eyint(r)[0],Ezint(r)[0]])*np.cos(2*np.pi*f_res*t+phi_RF)
    except:
        E = np.array([0,0,0])
    return E

## Checking Error to be $\mathcal{O}(1e-16)$

In [7]:
ERint, EZint, BTint, rr, zz, Emag = trueSuperfishFields("../../superfish/1T1.T7")

R = rr[4]
Z = zz[4]
th = np.pi/6

r = np.array([R*np.cos(th), R*np.sin(th), Z])
E_true = Efld(r,1e-9)
print(E_true)

ERint, EZint, BTint, rr, zz, Emag = toySuperfishFields("../../superfish/1T1.T7")
R = rr[4]
Z = zz[4]
th = np.pi/6

r = np.array([R*np.cos(th), R*np.sin(th), Z])
E_toy = Efld(r,1e-9)
print(E_toy)

0.0 0.29270745000000004 300 0.0 0.05 50
[1.98638778e+02 1.01793844e+02 1.56747123e+05]
0.0 0.29270745000000004 300 0.0 0.05 50
[1.98638778e+02 1.01793844e+02 1.56747123e+05]


In [8]:
E_diff = E_toy - E_true
E_diff

array([ 0.00000000e+00,  0.00000000e+00, -5.82076609e-11])

In [9]:
SCfac = 1.0
CST_flag = 1.0

#phi_RF = np.pi/2
dt = 1e-11 #1.0/(71*f_res)
Nt = 150 #)
t = np.linspace(0,Nt*dt,Nt,endpoint=False)
f_res = 1.3e9
Amp_fac = 28.836*(10/61.33) # 6.3 MV/m
phi_RF = np.pi/2+45/180*np.pi
print(Efld(np.flip(r),0))

[-9.10653686e+03 -5.25766151e+03 -7.03131138e+06]


Seems right to me... but I am still a bit weary about not implementing the edge cases in my interpolator. Do we not need to worry about those when going off of real data?

The Superfish outputs are:
```[1.98638778e+02 1.01793844e+02 1.56747123e+05]``` and  ```[-9.10653686e+03 -5.25766151e+03 -7.03131138e+06]```, which match exactly. This indicates that either:
- I correctly implemented the interpolator (which I hope is true!)
- Black magic caused the values to match nicely